# Data snooping Notebook

_Loren Champlin_

My notebook created for doing time series analysis on our WM indicators.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
import numpy as np
import pandas as pd
from delphi.db import engine
import random as rm
import delphi.evaluation_port as EN
import warnings
warnings.filterwarnings("ignore")
import logging
logging.getLogger().setLevel(logging.CRITICAL)
import time
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import matplotlib.patches as mpatches
import matplotlib.ticker as ticker
import traces
import datetime
import glob
import sys
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import pmdarima as pm

In [ ]:
def parse_iso_datetime(value):
    return datetime.datetime.strptime(value, "%Y-%m")

In [ ]:
def read_all(pattern):
    """Read all of the CSVs in a directory matching the filename pattern
    as TimeSeries.

    """
    result = []
    for filename in glob.iglob(pattern):
        print('reading', filename, file=sys.stderr)
        ts = traces.TimeSeries.from_csv(
            filename,
            time_column=0,
            time_transform=parse_iso_datetime,
            value_column=1,
            value_transform=float,
            default=0,
        )
        result.append(ts)
    return result

In [ ]:
def test_stationarity(timeseries):
    
    #Determing rolling statistics
    rolmean = timeseries.rolling(12).mean()
    rolstd = timeseries.rolling(12).std()
#Plot rolling statistics:
    plt.plot(timeseries, color='blue',label='Original')
    plt.plot(rolmean, color='red', label='Rolling Mean')
    plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show()
    #Perform Dickey-Fuller test:
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)

In [ ]:
ts_list = read_all("/Users/loren_champlin/Desktop/processed_data_IPC_and_migration/most_frequent_county_IPC_jonglei.csv")

ipc_ts = ts_list[0]

ipc_ts_true = ts_list[0]

#ipc_ts
ipc_df = ipc_ts.moving_average(2629754,pandas=True)


In [ ]:
date = []
year = 2009
month = 7
for i in range(120):
    date.append(f"{year}-{month}")
    if month == 12:
        month = 1
        year = year + 1
    else:
        month = month + 1


ipc_df = pd.DataFrame(ipc_df.values.round().reshape(-1,1), columns=["Most Frequent County IPC in Jonglei"])

ipc_df['Month'] = date

con=ipc_df['Month']
ipc_df['Month']=pd.to_datetime(ipc_df['Month'])
ipc_df.set_index('Month', inplace=True)
#check datatype of index
ipc_df.index

ipc_ts = ipc_df['Most Frequent County IPC in Jonglei']

sns.set(rc={"figure.figsize": (20, 10)}, style="whitegrid")

ax = sns.lineplot(data=ipc_ts, sort=False, markers=["o"], linewidth=3,markersize=10)
ax.set_title('Most Frequent County IPC in Jonglei (with imputation)')

In [ ]:
ipc_df = pd.DataFrame(ipc_ts_true, columns=["Month","Most Frequent County IPC in Jonglei"])

con=ipc_df['Month']
ipc_df['Month']=pd.to_datetime(ipc_df['Month'])
ipc_df.set_index('Month', inplace=True)
#check datatype of index
ipc_df.index

ipc_ts_true = ipc_df['Most Frequent County IPC in Jonglei']

sns.set(rc={"figure.figsize": (20, 10)}, style="whitegrid")

ax = sns.lineplot(data=ipc_ts_true, sort=False, markers=["o"], linewidth=3,markersize=10)
ax.set_title('Most Frequent County IPC in Jonglei')

In [ ]:
test_stationarity(ipc_ts)

In [ ]:
decomposition = seasonal_decompose(ipc_ts)

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

plt.subplot(411)
plt.plot(ipc_ts, label='Original')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend, label='Trend')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal, label='Seasonal')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(residual, label='Residual')
plt.legend(loc='best')

In [ ]:
fig, axes = plt.subplots(3, 2, sharex=True)
axes[0, 0].plot(ipc_ts.values); axes[0, 0].set_title('Original Series')
plot_acf(ipc_ts.values, ax=axes[0, 1])

axes[1, 0].plot(ipc_ts.diff().values); axes[1, 0].set_title('1st Order Differencing')
plot_acf(ipc_ts.diff().dropna().values, ax=axes[1, 1])

axes[2, 0].plot(ipc_ts.diff().diff().values); axes[2, 0].set_title('2nd Order Differencing')
plot_acf(ipc_ts.diff().diff().dropna().values, ax=axes[2, 1])

plt.show()

In [ ]:
model = ARIMA(ipc_ts, order=(0,1,1))
model_fit = model.fit(disp=0,trend='nc')
print(model_fit.summary())

In [ ]:
residuals = pd.DataFrame(model_fit.resid)
fig, ax = plt.subplots(1,2)
residuals.plot(title="Residuals", ax=ax[0])
residuals.plot(kind='kde', title='Density', ax=ax[1])
plt.show()

In [ ]:
model_fit.plot_predict(dynamic=False)
plt.show()

In [ ]:
#len(ipc_ts)*.75, len(ipc_ts)*.25

train = ipc_ts[:90]
test = ipc_ts[89:]



In [ ]:
model_t = ARIMA(train, order=(0,1,1))
model_fit_t = model_t.fit(disp=0,trend='nc')
print(model_fit_t.summary())

In [ ]:
residuals = pd.DataFrame(model_fit_t.resid)
fig, ax = plt.subplots(1,2)
residuals.plot(title="Residuals", ax=ax[0])
residuals.plot(kind='kde', title='Density', ax=ax[1])
plt.show()

In [ ]:
fc, se, conf = model_fit_t.forecast(31, alpha=0.05)

fc = fc.round()

In [ ]:
fc_series = pd.Series(fc, index=test.index)
fc_series = fc_series.round()
lower_series = pd.Series(conf[:, 0], index=test.index)
upper_series = pd.Series(conf[:, 1], index=test.index)
lower_series = lower_series.round()
upper_series = upper_series.round()
lower_series[lower_series < 1] = 1
lower_series[lower_series > 5] = 5
upper_series[upper_series < 1] = 1
upper_series[upper_series > 5] = 5

In [ ]:
plt.figure(figsize=(12,5), dpi=100)
plt.plot(train, label='training')
plt.plot(test, label='actual')
plt.plot(fc_series, label='forecast')
plt.fill_between(lower_series.index, lower_series, upper_series, 
                 color='k', alpha=.15)
plt.title('Most Frequent County IPC in Jonglei (Forecast vs Actuals)')
plt.legend(loc='upper left', fontsize=8)
plt.show()

In [ ]:
def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    mins = np.amin(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    maxs = np.amax(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    minmax = 1 - np.mean(mins/maxs)             # minmax
    acf1 = acf(forecast-actual)[1]                      # ACF1
    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse, 'acf1':acf1, 
            'corr':corr, 'minmax':minmax})

In [ ]:
forecast_accuracy(fc, test.values)

In [ ]:
model = pm.auto_arima(ipc_ts, start_p=0, start_q=0,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=4, max_q=4, # maximum p and q
                      m=12,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=None, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

print(model.summary())

In [ ]:
train = ipc_ts[:90]
test = ipc_ts[89:]
smodel = pm.auto_arima(train, start_p=0, start_q=0,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=4, max_q=4, # maximum p and q
                      m=12,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=True,   # No Seasonality
                      start_P=0, 
                      D=None,
                      max_D = 3,
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

print(smodel.summary())

In [ ]:
residuals = pd.DataFrame(smodel.resid())
fig, ax = plt.subplots(1,2)
residuals.plot(title="Residuals", ax=ax[0])
residuals.plot(kind='kde', title='Density', ax=ax[1])
plt.show()

In [ ]:
n_periods = 31
fitted, confint = smodel.predict(n_periods=n_periods, return_conf_int=True)
fitted = fitted.round()
index_of_fc = pd.date_range(train.index[-1], periods = n_periods, freq='MS')
fitted_series = pd.Series(fitted, index=index_of_fc)
fitted_series = fitted_series.round()
lower_series = pd.Series(confint[:, 0], index=index_of_fc)
upper_series = pd.Series(confint[:, 1], index=index_of_fc)
lower_series = lower_series.round()
upper_series = upper_series.round()
lower_series[lower_series < 1] = 1
lower_series[lower_series > 5] = 5
upper_series[upper_series < 1] = 1
upper_series[upper_series > 5] = 5

In [ ]:
plt.figure(figsize=(12,5), dpi=100)
plt.plot(train, label='training')
plt.plot(test, label='actual')
plt.plot(fitted_series, label='forecast')
plt.fill_between(lower_series.index, lower_series, upper_series, 
                 color='k', alpha=.15)
plt.title('Most Frequent County IPC in Jonglei (Forecast vs Actuals)')
plt.legend(loc='upper left', fontsize=8)
plt.show()

In [ ]:
forecast_accuracy(fitted, test.values)

In [ ]:
ts_list = read_all("/Users/loren_champlin/Desktop/processed_data_IPC_and_migration/new_asylum_seeking_applicants_south_sudan.csv")

as_ts = ts_list[0]

as_ts_true = ts_list[0]

#ipc_ts
as_df = as_ts.moving_average(2600837,pandas=True)


In [ ]:
date = []
year = 2011
month = 7
for i in range(88):
    date.append(f"{year}-{month}")
    if month == 12:
        month = 1
        year = year + 1
    else:
        month = month + 1


as_df = pd.DataFrame(as_df.values.round().reshape(-1,1), columns=["New Asylum Seeking applicants in South Sudan"])

as_df['Month'] = date

con=as_df['Month']
as_df['Month']=pd.to_datetime(as_df['Month'])
as_df.set_index('Month', inplace=True)
#check datatype of index
as_df.index

as_ts = as_df["New Asylum Seeking applicants in South Sudan"]

sns.set(rc={"figure.figsize": (20, 10)}, style="whitegrid")

ax = sns.lineplot(data=as_ts, sort=False, markers=["o"], linewidth=3,markersize=10)
ax.set_title("New Asylum Seeking applicants in South Sudan (with Smoothing)")

In [ ]:
as_df = pd.DataFrame(as_ts_true, columns=["Month","New Asylum Seeking applicants in South Sudan"])

con=as_df['Month']
as_df['Month']=pd.to_datetime(as_df['Month'])
as_df.set_index('Month', inplace=True)
#check datatype of index
as_df.index

as_ts_true = as_df['New Asylum Seeking applicants in South Sudan']

sns.set(rc={"figure.figsize": (20, 10)}, style="whitegrid")

ax = sns.lineplot(data=as_ts_true, sort=False, markers=["o"], linewidth=3,markersize=10)
ax.set_title('New Asylum Seeking applicants in South Sudan')

In [ ]:
test_stationarity(as_ts)

In [ ]:
decomposition = seasonal_decompose(as_ts)

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

plt.subplot(411)
plt.plot(as_ts, label='Original')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend, label='Trend')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal, label='Seasonal')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(residual, label='Residual')
plt.legend(loc='best')

In [ ]:
fig, axes = plt.subplots(3, 2, sharex=True)
axes[0, 0].plot(as_ts.values); axes[0, 0].set_title('Original Series')
plot_acf(as_ts.values, ax=axes[0, 1])

axes[1, 0].plot(as_ts.diff().values); axes[1, 0].set_title('1st Order Differencing')
plot_acf(as_ts.diff().dropna().values, ax=axes[1, 1])

axes[2, 0].plot(as_ts.diff().diff().values); axes[2, 0].set_title('2nd Order Differencing')
plot_acf(as_ts.diff().diff().dropna().values, ax=axes[2, 1])

plt.show()

In [ ]:
model = ARIMA(as_ts, order=(2,1,1))
model_fit = model.fit(disp=0, trend='nc')
print(model_fit.summary())

In [ ]:
residuals = pd.DataFrame(model_fit.resid)
fig, ax = plt.subplots(1,2)
residuals.plot(title="Residuals", ax=ax[0])
residuals.plot(kind='kde', title='Density', ax=ax[1])
plt.show()

In [ ]:
model_fit.plot_predict(dynamic=False)
plt.show()

In [ ]:
#len(as_ts)*.75, len(as_ts)*.25

train = as_ts[:66]
test = as_ts[65:]

In [ ]:
model_t = ARIMA(train, order=(2,1,1))
model_fit_t = model_t.fit(disp=0, trend='nc')
print(model_fit_t.summary())

In [ ]:
residuals = pd.DataFrame(model_fit_t.resid)
fig, ax = plt.subplots(1,2)
residuals.plot(title="Residuals", ax=ax[0])
residuals.plot(kind='kde', title='Density', ax=ax[1])
plt.show()

In [ ]:
fc, se, conf = model_fit_t.forecast(23, alpha=0.05)

In [ ]:
fc_series = pd.Series(fc, index=test.index)
lower_series = pd.Series(conf[:, 0], index=test.index)
upper_series = pd.Series(conf[:, 1], index=test.index)

In [ ]:
plt.figure(figsize=(12,5), dpi=100)
plt.plot(train, label='training')
plt.plot(test, label='actual')
plt.plot(fc_series, label='forecast')
plt.fill_between(lower_series.index, lower_series, upper_series, 
                 color='k', alpha=.15)
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()

In [ ]:
forecast_accuracy(fc, test.values)

In [ ]:
model = pm.auto_arima(as_ts, start_p=0, start_q=0,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=4, max_q=4, # maximum p and q
                      m=12,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=None, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

print(model.summary())

In [ ]:
train = as_ts[:66]
test = as_ts[65:]
smodel = pm.auto_arima(train, start_p=0, start_q=0,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=4, max_q=4, # maximum p and q
                      m=12,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=True,   # No Seasonality
                      start_P=0, 
                      D=None,
                      max_D = 3,
                      max_P = 3,
                      max_Q = 3,
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True,
                      with_intercept=True)

print(smodel.summary())

In [ ]:
residuals = pd.DataFrame(smodel.resid())
fig, ax = plt.subplots(1,2)
residuals.plot(title="Residuals", ax=ax[0])
residuals.plot(kind='kde', title='Density', ax=ax[1])
plt.show()

In [ ]:
n_periods = 23
fitted, confint = smodel.predict(n_periods=n_periods, return_conf_int=True)
index_of_fc = pd.date_range(train.index[-1], periods = n_periods, freq='MS')
fitted_series = pd.Series(fitted, index=index_of_fc)
lower_series = pd.Series(confint[:, 0], index=index_of_fc)
upper_series = pd.Series(confint[:, 1], index=index_of_fc)

In [ ]:
plt.figure(figsize=(12,5), dpi=100)
plt.plot(train, label='training')
plt.plot(test, label='actual')
plt.plot(fitted_series, label='forecast')
plt.fill_between(lower_series.index, lower_series, upper_series, 
                 color='k', alpha=.15)
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()

In [ ]:
forecast_accuracy(fitted, test.values)